In [1]:
# -*- coding: utf-8 -*-
import os
import numpy as np
import cv2
import dlib
import time
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Load dataset

In [2]:
data = pd.read_pickle('faces_full.pkl')
detector = dlib.get_frontal_face_detector()
p = "shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor (p)
face_rec_model_path = 'dlib_face_recognition_resnet_model_v1.dat'
facerec = dlib.face_recognition_model_v1(face_rec_model_path)

In [6]:
list(data['file'][:10])

['D://Data\\video\\8.29张江人工智能岛\\29号\\摄像迟\\BPAV\\CLPR\\444_0340_01\\3.0.jpg',
 'D://Data\\video\\8.29张江人工智能岛\\29号\\摄像迟\\BPAV\\CLPR\\444_0340_01\\6.0.jpg',
 'D://Data\\video\\8.29张江人工智能岛\\29号\\摄像迟\\BPAV\\CLPR\\444_0346_01\\0.0.jpg',
 'D://Data\\video\\8.29张江人工智能岛\\29号\\摄像迟\\BPAV\\CLPR\\444_0346_01\\1.0.jpg',
 'D://Data\\video\\8.29张江人工智能岛\\29号\\摄像迟\\BPAV\\CLPR\\444_0346_01\\1.0.jpg',
 'D://Data\\video\\8.29张江人工智能岛\\29号\\摄像迟\\BPAV\\CLPR\\444_0346_01\\1.0.jpg',
 'D://Data\\video\\8.29张江人工智能岛\\29号\\摄像迟\\BPAV\\CLPR\\444_0346_01\\2.0.jpg',
 'D://Data\\video\\8.29张江人工智能岛\\29号\\摄像迟\\BPAV\\CLPR\\444_0346_01\\2.0.jpg',
 'D://Data\\video\\8.29张江人工智能岛\\29号\\摄像迟\\BPAV\\CLPR\\444_0346_01\\3.0.jpg',
 'D://Data\\video\\8.29张江人工智能岛\\29号\\摄像迟\\BPAV\\CLPR\\444_0346_01\\4.0.jpg']

In [7]:
data['file'][59]

'D://Data\\video\\8.29张江人工智能岛\\29号\\摄像迟\\BPAV\\CLPR\\444_0351_01\\1.0.jpg'

In [ ]:
def get_faces(file):
    """
        This function has the same functionality as in 2-Get_face_descriptor.ipynb
        It returns a pandas dataframe contains the face decriptors
    """
    faces = []
    image = cv2.imdecode(np.fromfile(file,dtype=np.uint8),-1)
    image = cv2. cvtColor (image, cv2.COLOR_BGR2RGB)
    gray = cv2. cvtColor (image, cv2.COLOR_RGB2GRAY)

    rects = detector (gray , 0)
    if len(rects) >=0:
        for rect in rects :
            shape = predictor (gray , rect )
            face_chip = dlib.get_face_chip(image, shape)
            face_descriptor = facerec.compute_face_descriptor(face_chip)
            item = [file, [rect.left(), rect.top(), rect.width(), rect.height()], list(face_descriptor)]
            faces.append(item)
    return pd.DataFrame(faces, columns=['file', 'rect', 'face_descriptor'])

In [ ]:
def rec_person(row):
    """
        This function takes a file's path (better absolute path) as argument and returns the image's information with 
        the most likely match
    """
    max_d = 1000
    person = None
    for i in range(len(data)):
        f = data.iloc[i]
        #print(f)
        if f.file == row.file:
            continue
        #print(row.face_descriptor)
        d = np.linalg.norm(np.array(f.face_descriptor) - np.array(row.face_descriptor))
        if d < max_d:
            max_d = d
            person = f
    if max_d<0.6:
        return([max_d,person])
    else:
        return None

In [ ]:
def rec_persons(row, group):
    """
        This function takes a file's path (better absolute path) as argument and returns all images with similar face.
        The result is a list of pandas series
    """
    for i in range(len(data)):
        f = data.iloc[i]
        #print(f)
        if f.file == row.file or f.distance < 0.3:
            continue
        #print(row.face_descriptor)
        d = np.linalg.norm(np.array(f.face_descriptor) - np.array(row.face_descriptor))
        if d < f.distance:
            data['distance'][i] = d
            data['group'][i] = group

In [ ]:
img = "D:\\Data\\8.29张江人工智能岛\\31号\\照片\\摄影  文\\100EOS5D\\0O8A8700.JPG" # Path to sample image for searching
img_info = get_faces(img)
img_info

In [ ]:
start = time.time()

for i in range(len(data)):
    
    rec_persons(img_info.iloc[i], i)

print(time.time()- start)

In [ ]:
data.groupby('file').count().sort_values(by='face_descriptor')

In [10]:
distance = []
with open("result.txt") as f:
    for line in f:
        distance.append(int(line))

In [13]:
data['group'] = distance

In [15]:
data.groupby('group').count()

,file,rect,face_descriptor
group,,,
2,35,35,35
18,11,11,11
33,1,1,1
40,18,18,18
41,3017,3017,3017
43,328,328,328
70,1,1,1
79,5,5,5
83,369,369,369


In [23]:
data.head()

,file,rect,face_descriptor,group
0,D://Data\video\8.29张江人工智能岛\29号\摄像迟\BPAV\CLPR\4...,"[925, 365, 73, 73]","[-0.10779663920402527, 0.10153406113386154, 0....",110
1,D://Data\video\8.29张江人工智能岛\29号\摄像迟\BPAV\CLPR\4...,"[813, 362, 87, 87]","[-0.060471706092357635, 0.036922015249729156, ...",41
2,D://Data\video\8.29张江人工智能岛\29号\摄像迟\BPAV\CLPR\4...,"[369, 231, 126, 125]","[-0.09775551408529282, 0.07954566180706024, 0....",110
3,D://Data\video\8.29张江人工智能岛\29号\摄像迟\BPAV\CLPR\4...,"[353, 238, 105, 105]","[-0.09728292375802994, 0.04980679973959923, 0....",110
4,D://Data\video\8.29张江人工智能岛\29号\摄像迟\BPAV\CLPR\4...,"[839, 176, 126, 125]","[-0.003757824655622244, 0.09656134992837906, 0...",83


In [17]:
matrix = []
with open('matrix.txt') as f:
    for line in f:
        t = []
        for d in line.split():
            t.append(float(d))
        matrix.append(t)

28304